<a href="https://colab.research.google.com/github/lucinnnal/Convolutional-Neural-Network/blob/main/2021311409_%EA%B9%80%EA%B8%B0%ED%91%9C_A_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from keras.datasets import mnist
from matplotlib import pyplot as plt

**axis=(a, b)는 지정한 두 축 a와 b를 따라 값을 합산하고, 나머지 축만 남김.
다차원 배열의 특정 부분을 처리할 때 유용**

In [20]:
class Conv3x3:

    def __init__(self, num_filters):

        self.num_filters = num_filters #filter 의 개수 == output의 차원
        self.filters = np.random.randn(num_filters, 3, 3) / 9 # initialize num_filters x 3 x 3

    def forward(self, input):

        self.last_input = input

        h, w = input.shape
        output = np.zeros((h-2, w-2, self.num_filters)) # write code here (1 line) -> assuming strde 1?

######write code here (3 lines)######
#
#        for i in range(None):
#            for j in range(None):
#
#
#####################################
        for i in range(output.shape[0]):
          for j in range(output.shape[1]):
            output[i,j,:] = np.sum(input[i:i+3, j:j+3] * self.filters, axis = (1,2))

        return output

class MaxPool2: #2x2 maxpooling

    def forward(self, input):

        self.last_input = input
        h, w, num_filters = input.shape

        output = np.zeros((h // 2, w // 2, num_filters)) #write code here (1 line)

######write code here (3 lines)######
#
#        for i in range(None):
#            for j in range(None):
#
#####################################
        for i in range(output.shape[0]):
          for j in range(output.shape[1]):
            output[i,j,:] = np.max(np.transpose(input[2*i:2*(i+1),2*j:2*(j+1),:], (2,0,1)), axis = (1,2))

        return output


class Softmax:
    def __init__(self, input_len, nodes):

        self.weights = np.random.randn(input_len, nodes) / input_len
        self.biases = np.zeros(nodes)


    def forward(self, input):
        self.last_input_shape = input.shape # Caching for backprop

        input = input.flatten() # input shape = (input_len,)
        self.last_input = input # Caching for backprop

        totals = np.dot(self.weights.T, input) + self.biases # write code here (1 line) -> totals shape : (10,)

        self.last_totals = totals # Caching for backprop

        exp_a = np.exp(totals) # write code here (1 line)
        return exp_a / np.sum(exp_a) # write code here (1 line) -> (10,)


    def backprop(self, dL_dOut, learn_rate):
        for i, gradient in enumerate(dL_dOut):
            if gradient == 0:
                continue # 정답 index에 대해서만 gradient 계산을 할 것임.

            # e^totals
            t_exp = np.exp(self.last_totals) # write code here (1 line)

            # Sum of all e^totals
            S = np.sum(t_exp) # write code here (1 line)

            # Gradient of out[i] against totals
            dOut_dt =  -1 * (t_exp / S) * (t_exp / S)[i]  # write code here (1 line) -> (10,1)
            dOut_dt[i] =  (t_exp / S)[i] * (1 - (t_exp / S)[i]) # write code here (1 line) (y_hat - y)

            # weights/biases/input gradient #
            # write code here (3 lines)
            dt_dw = self.last_input
            dt_db = np.ones(10)
            dt_dInputs = self.weights # v
            ################################

            # compute gradient
            # write code here (4 lines)
            dL_dt = dL_dOut[i] * dOut_dt # ->(10,)
            dL_dw = np.dot(dt_dw[:, np.newaxis], dL_dt[:, np.newaxis].T)
            dL_db = dL_dt * dt_db # (10,)
            dL_dInputs = np.dot(dt_dInputs, dL_dt[:,np.newaxis]) # (input_len, 1)

            # update weights/biases
            # write code here (2 lines)
            self.weights -= learn_rate * dL_dw
            self.biases -= learn_rate * dL_db
            return dL_dInputs.reshape(self.last_input_shape)

# Test (수정 X)

In [24]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images[:1000]
train_labels = train_labels[:1000]

conv = Conv3x3(8)
pool = MaxPool2()
softmax = Softmax(13 * 13 * 8, 10)

def forward(image, label):
    out = conv.forward((image / 255) - 0.5)
    out = pool.forward(out)
    out = softmax.forward(out)

    loss = -np.log(out[label]) # Cross Entropy
    acc = 1 if np.argmax(out) == label else 0 # 하나 하나씩 학습시키는 것임

    return out, loss, acc

def train(im, label, lr = .005):
    # Forward
    out, loss, acc = forward(im, label)

    # Calculate initial gradient
    gradient = np.zeros(10)
    gradient[label] = -1/out[label]

    # Backprop
    gradient = softmax.backprop(gradient, lr)

    return loss, acc

loss = 0
num_correct = 0
for i, (im, label) in enumerate(zip(train_images, train_labels)):
    if i % 100 == 99:
        print(
            '[Step %d] Past 100 steps: Average Loss %.3f | Accuracy: %d%%' %
            (i + 1, loss / 100, num_correct)
        )
        loss = 0
        num_correct = 0

    l, acc = train(im, label)
    loss += l
    num_correct += acc

[Step 100] Past 100 steps: Average Loss 2.217 | Accuracy: 20%
[Step 200] Past 100 steps: Average Loss 2.101 | Accuracy: 35%
[Step 300] Past 100 steps: Average Loss 1.944 | Accuracy: 51%
[Step 400] Past 100 steps: Average Loss 1.794 | Accuracy: 69%
[Step 500] Past 100 steps: Average Loss 1.711 | Accuracy: 60%
[Step 600] Past 100 steps: Average Loss 1.741 | Accuracy: 54%
[Step 700] Past 100 steps: Average Loss 1.656 | Accuracy: 64%
[Step 800] Past 100 steps: Average Loss 1.526 | Accuracy: 70%
[Step 900] Past 100 steps: Average Loss 1.441 | Accuracy: 76%
[Step 1000] Past 100 steps: Average Loss 1.414 | Accuracy: 75%


# 수고하셨습니다 :)